In [ ]:
import requests
from datetime import datetime, timedelta
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from difflib import SequenceMatcher

## Parameters

In [ ]:
# API Configuration
fabric_gps_api_url = "https://fabric-gps.com/api/releases"
modified_within_days = 90
page_size = 200
include_planned = True
include_shipped = True

# Alert Thresholds
ALERT_DAYS_THRESHOLD = 90  # Alert if preview active >90 days
LOW_CONFIDENCE_THRESHOLD = 0.5  # Alert if similarity score <0.5
SIMILARITY_MATCH_THRESHOLD = 0.3  # Minimum similarity to consider a match

# Alert Severity Levels
SEVERITY_INFO = "Info"
SEVERITY_WARNING = "Warning"
SEVERITY_CRITICAL = "Critical"

---
# Part 1: Fetch Feature Releases from Fabric GPS API

## Step 1.1: Fetch from API

In [ ]:
def fetch_all_fabric_gps_releases(base_url, page_size=200, modified_within_days=None, include_planned=True, include_shipped=True):
    """Fetch all releases from Fabric GPS API with pagination"""
    all_releases = []
    page = 1
    
    print(f"🔄 Fetching from Fabric GPS API: {base_url}")
    
    while True:
        try:
            params = {"page": page, "page_size": page_size}
            
            if modified_within_days and modified_within_days <= 30:
                params["modified_within_days"] = modified_within_days
            
            response = requests.get(base_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            releases = data.get("data", [])
            if not releases:
                break
            
            # Filter by release_status
            filtered_releases = []
            for release in releases:
                status = release.get("release_status", "")
                if status == "Planned" and not include_planned:
                    continue
                if status == "Shipped" and not include_shipped:
                    continue
                filtered_releases.append(release)
            
            all_releases.extend(filtered_releases)
            
            pagination = data.get("pagination", {})
            has_next = pagination.get("has_next", False)
            total_items = pagination.get("total_items", 0)
            
            print(f"  → Page {page}: {len(filtered_releases)} releases (Total: {len(all_releases)}/{total_items})")
            
            if not has_next:
                break
            
            page += 1
            
        except Exception as e:
            print(f"  ❌ Error fetching page {page}: {e}")
            break
    
    print(f"✅ Total releases fetched: {len(all_releases)}")
    return all_releases

releases_data = fetch_all_fabric_gps_releases(
    fabric_gps_api_url, 
    page_size=page_size,
    modified_within_days=modified_within_days if modified_within_days <= 30 else None,
    include_planned=include_planned,
    include_shipped=include_shipped
)

## Step 1.2: Transform to Schema

In [ ]:
def transform_fabric_gps_to_fuam(releases):
    """Transform Fabric GPS release data to FUAM schema"""
    transformed = []
    
    workload_mapping = {
        "Power BI": "Power BI",
        "Data Factory": "Data Factory",
        "Data Engineering": "Data Engineering",
        "Data Science": "Data Science",
        "Data Warehouse": "Data Warehouse",
        "Real-Time Intelligence": "Real-Time Intelligence",
        "OneLake": "Data Engineering",
        "Administration, Governance and Security": "Governance",
        "Cosmos DB (NoSQL)": "Cosmos DB"
    }
    
    for release in releases:
        try:
            release_date_str = release.get("release_date")
            release_date = datetime.strptime(release_date_str, "%Y-%m-%d") if release_date_str else None
        except:
            release_date = None
        
        try:
            last_modified_str = release.get("last_modified")
            last_modified = datetime.strptime(last_modified_str, "%Y-%m-%d") if last_modified_str else datetime.now()
        except:
            last_modified = datetime.now()
        
        release_type = release.get("release_type", "")
        is_preview = "preview" in release_type.lower()
        product_name = release.get("product_name", "Unknown")
        workload = workload_mapping.get(product_name, product_name)
        
        feature = {
            "feature_id": release.get("release_item_id"),
            "feature_name": release.get("feature_name", "Unknown"),
            "feature_description": release.get("feature_description"),
            "workload": workload,
            "product_name": product_name,
            "release_date": release_date,
            "release_type": release_type,
            "release_status": release.get("release_status", "Unknown"),
            "is_preview": is_preview,
            "is_planned": release.get("release_status") == "Planned",
            "is_shipped": release.get("release_status") == "Shipped",
            "last_modified": last_modified,
            "source_url": f"https://fabric-gps.com/api/releases?release_item_id={release.get('release_item_id')}",
            "source": "Fabric GPS API",
            "extracted_date": datetime.now()
        }
        
        transformed.append(feature)
    
    return transformed

print("🔄 Transforming to schema...")
features_data = transform_fabric_gps_to_fuam(releases_data)

preview_count = sum(1 for f in features_data if f["is_preview"])
planned_count = sum(1 for f in features_data if f["is_planned"])
shipped_count = sum(1 for f in features_data if f["is_shipped"])

print(f"✅ Transformed {len(features_data)} features")
print(f"   Preview: {preview_count} | Planned: {planned_count} | Shipped: {shipped_count}")

## Step 1.3: Write Feature Releases to Delta

In [ ]:
schema = StructType([
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("feature_description", StringType(), True),
    StructField("workload", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("release_date", TimestampType(), True),
    StructField("release_type", StringType(), True),
    StructField("release_status", StringType(), True),
    StructField("is_preview", BooleanType(), False),
    StructField("is_planned", BooleanType(), False),
    StructField("is_shipped", BooleanType(), False),
    StructField("last_modified", TimestampType(), False),
    StructField("source_url", StringType(), True),
    StructField("source", StringType(), True),
    StructField("extracted_date", TimestampType(), False)
])

df_features = spark.createDataFrame(features_data, schema=schema)

table_name = "feature_releases_roadmap"
table_path = f"Tables/{table_name}"

print(f"🔄 Writing to {table_name}...")

try:
    from delta.tables import DeltaTable
    
    if DeltaTable.isDeltaTable(spark, table_path):
        print("  → Performing MERGE (upsert)...")
        
        delta_table = DeltaTable.forPath(spark, table_path)
        
        delta_table.alias("target").merge(
            df_features.alias("source"),
            "target.feature_id = source.feature_id"
        ).whenMatchedUpdate(
            condition="source.last_modified > target.last_modified",
            set={
                "feature_name": "source.feature_name",
                "feature_description": "source.feature_description",
                "workload": "source.workload",
                "product_name": "source.product_name",
                "release_date": "source.release_date",
                "release_type": "source.release_type",
                "release_status": "source.release_status",
                "is_preview": "source.is_preview",
                "is_planned": "source.is_planned",
                "is_shipped": "source.is_shipped",
                "last_modified": "source.last_modified",
                "source_url": "source.source_url",
                "extracted_date": "source.extracted_date"
            }
        ).whenNotMatchedInsertAll().execute()
        
        print("  ✅ MERGE completed")
    else:
        print("  → Creating new table...")
        df_features.write.format("delta").mode("overwrite").save(table_path)
        print("  ✅ Table created")
    
    final_count = spark.read.format("delta").load(table_path).count()
    print(f"✅ Total records in {table_name}: {final_count}")
    
except Exception as e:
    print(f"❌ Error writing to Delta: {e}")
    raise

---
# Part 2: Detect Activated Preview Features

## Step 2.1: Load Data Sources

In [ ]:
print("\n🔄 Loading preview features...")
df_preview_features = spark.read.format("delta").load(table_path).filter(F.col("is_preview") == True)
preview_count = df_preview_features.count()
print(f"✅ Loaded {preview_count} preview features")

print("\n🔄 Loading tenant settings...")
try:
    df_tenant_settings = spark.read.format("delta").load("Tables/tenant_settings")
    settings_count = df_tenant_settings.count()
    print(f"✅ Loaded {settings_count} tenant settings")
except Exception as e:
    print(f"⚠️ Warning: Could not load tenant_settings table: {e}")
    print("   Skipping preview feature detection...")
    df_tenant_settings = None

## Step 2.2: Map Settings to Features

In [ ]:
def similarity_score(a, b):
    """Calculate similarity between two strings (0-1)"""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def map_settings_to_features(df_settings, df_features, threshold=SIMILARITY_MATCH_THRESHOLD):
    """Map tenant settings to preview features based on name similarity"""
    
    features_list = df_features.select("feature_id", "feature_name", "workload", "release_date", "release_status", "source_url").collect()
    settings_list = df_settings.select("settingName", "enabled", "delegateToDomain", "delegateToWorkspace", "delegateToCapacity").collect()
    
    matches = []
    
    for setting in settings_list:
        setting_name = setting["settingName"]
        
        if not setting["enabled"]:
            continue
        
        best_match = None
        best_score = 0.0
        
        for feature in features_list:
            feature_name = feature["feature_name"]
            
            # Calculate similarity
            score = similarity_score(setting_name, feature_name)
            
            # Boost score for common words
            setting_words = set(setting_name.lower().split())
            feature_words = set(feature_name.lower().split())
            common_words = setting_words & feature_words
            
            if common_words:
                score += len(common_words) * 0.1
            
            if score > best_score and score > threshold:
                best_score = score
                best_match = feature
        
        if best_match:
            # Calculate days since release
            days_since = None
            if best_match["release_date"]:
                days_since = (datetime.now() - best_match["release_date"]).days
            
            # Check if any delegation is enabled
            is_delegated = setting["delegateToDomain"] or setting["delegateToWorkspace"] or setting["delegateToCapacity"]
            
            matches.append({
                "setting_name": setting_name,
                "feature_id": best_match["feature_id"],
                "feature_name": best_match["feature_name"],
                "workload": best_match["workload"],
                "similarity_score": best_score,
                "is_enabled": setting["enabled"],
                "delegate_to_tenant": is_delegated,
                "detected_date": datetime.now(),
                "release_date": best_match["release_date"],
                "release_status": best_match["release_status"],
                "source_url": best_match["source_url"],
                "days_since_release": days_since
            })
    
    return matches

if df_tenant_settings:
    print("🔄 Mapping settings to preview features...")
    matches = map_settings_to_features(df_tenant_settings, df_preview_features)
    print(f"✅ Found {len(matches)} activated preview features")
else:
    matches = []

## Step 2.3: Write Active Previews to Delta

In [ ]:
if matches:
    schema_active = StructType([
        StructField("setting_name", StringType(), False),
        StructField("feature_id", StringType(), False),
        StructField("feature_name", StringType(), False),
        StructField("workload", StringType(), True),
        StructField("similarity_score", DoubleType(), False),
        StructField("is_enabled", BooleanType(), False),
        StructField("delegate_to_tenant", BooleanType(), True),
        StructField("detected_date", TimestampType(), False),
        StructField("release_date", TimestampType(), True),
        StructField("release_status", StringType(), True),
        StructField("source_url", StringType(), True),
        StructField("days_since_release", IntegerType(), True)
    ])
    
    df_active_previews = spark.createDataFrame(matches, schema=schema_active)
    
    table_name_active = "preview_features_active"
    table_path_active = f"Tables/{table_name_active}"
    
    print(f"🔄 Writing to {table_name_active}...")
    
    try:
        if DeltaTable.isDeltaTable(spark, table_path_active):
            print("  → Performing MERGE...")
            
            delta_table = DeltaTable.forPath(spark, table_path_active)
            
            delta_table.alias("target").merge(
                df_active_previews.alias("source"),
                "target.feature_id = source.feature_id AND target.setting_name = source.setting_name"
            ).whenMatchedUpdate(
                set={
                    "is_enabled": "source.is_enabled",
                    "detected_date": "source.detected_date",
                    "days_since_release": "source.days_since_release",
                    "similarity_score": "source.similarity_score"
                }
            ).whenNotMatchedInsertAll().execute()
            
            print("  ✅ MERGE completed")
        else:
            print("  → Creating new table...")
            df_active_previews.write.format("delta").mode("overwrite").save(table_path_active)
            print("  ✅ Table created")
        
        final_count = spark.read.format("delta").load(table_path_active).count()
        print(f"✅ Total activated previews: {final_count}")
        
    except Exception as e:
        print(f"❌ Error writing active previews: {e}")
        raise
else:
    print("ℹ️ No activated preview features to write")
    df_active_previews = None

---
# Part 3: Generate Feature Alerts

## Step 3.1: Load Historical Alerts

In [ ]:
table_name_alerts = "feature_alerts"
table_path_alerts = f"Tables/{table_name_alerts}"

try:
    if DeltaTable.isDeltaTable(spark, table_path_alerts):
        print("🔄 Loading historical alerts...")
        df_historical = spark.read.format("delta").load(table_path_alerts)
        
        # Get already alerted feature_ids + alert_type combinations to avoid duplicate alerts
        alerted_combos = set([
            (row["feature_id"], row["alert_type"]) 
            for row in df_historical.select("feature_id", "alert_type").distinct().collect()
        ])
        
        print(f"✅ Loaded {len(alerted_combos)} existing alert combinations")
    else:
        alerted_combos = set()
        print("ℹ️ No historical alerts found (first run)")
except Exception as e:
    alerted_combos = set()
    print(f"ℹ️ No historical alerts (first run): {e}")

## Step 3.2: Generate Alerts

In [ ]:
def generate_alerts(matches, alerted_combos):
    """Generate alerts based on business rules"""
    alerts = []
    
    for match in matches:
        feature_id = match["feature_id"]
        feature_name = match["feature_name"]
        workload = match["workload"]
        days_since_release = match["days_since_release"]
        similarity_score = match["similarity_score"]
        setting_name = match["setting_name"]
        
        # Rule 1: NEW PREVIEW FEATURE ACTIVATED
        alert_type = "New Preview Activated"
        if (feature_id, alert_type) not in alerted_combos:
            alerts.append({
                "alert_id": f"NEW_{feature_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}",
                "feature_id": feature_id,
                "feature_name": feature_name,
                "workload": workload,
                "alert_type": alert_type,
                "severity": SEVERITY_INFO,
                "message": f"New preview feature '{feature_name}' activated (Setting: {setting_name})",
                "setting_name": setting_name,
                "similarity_score": similarity_score,
                "days_since_release": days_since_release,
                "alert_date": datetime.now(),
                "acknowledged": False,
                "acknowledged_date": None,
                "acknowledged_by": None
            })
        
        # Rule 2: LONG-RUNNING PREVIEW (>90 days)
        if days_since_release and days_since_release > ALERT_DAYS_THRESHOLD:
            alert_type = "Long-Running Preview"
            if (feature_id, alert_type) not in alerted_combos:
                alerts.append({
                    "alert_id": f"LONGRUN_{feature_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}",
                    "feature_id": feature_id,
                    "feature_name": feature_name,
                    "workload": workload,
                    "alert_type": alert_type,
                    "severity": SEVERITY_WARNING,
                    "message": f"Preview feature '{feature_name}' active for {days_since_release} days. Review for GA transition.",
                    "setting_name": setting_name,
                    "similarity_score": similarity_score,
                    "days_since_release": days_since_release,
                    "alert_date": datetime.now(),
                    "acknowledged": False,
                    "acknowledged_date": None,
                    "acknowledged_by": None
                })
        
        # Rule 3: LOW CONFIDENCE MATCH
        if similarity_score < LOW_CONFIDENCE_THRESHOLD:
            alert_type = "Low Confidence Match"
            if (feature_id, alert_type) not in alerted_combos:
                alerts.append({
                    "alert_id": f"LOWCONF_{feature_id}_{datetime.now().strftime('%Y%m%d%H%M%S')}",
                    "feature_id": feature_id,
                    "feature_name": feature_name,
                    "workload": workload,
                    "alert_type": alert_type,
                    "severity": SEVERITY_CRITICAL,
                    "message": f"Low confidence match ({similarity_score:.2f}) between setting '{setting_name}' and feature '{feature_name}'. Manual review recommended.",
                    "setting_name": setting_name,
                    "similarity_score": similarity_score,
                    "days_since_release": days_since_release,
                    "alert_date": datetime.now(),
                    "acknowledged": False,
                    "acknowledged_date": None,
                    "acknowledged_by": None
                })
    
    return alerts

if matches:
    print("🔄 Generating alerts...")
    alerts_data = generate_alerts(matches, alerted_combos)
    print(f"✅ Generated {len(alerts_data)} new alerts")
    
    if alerts_data:
        for alert in alerts_data[:3]:
            print(f"  [{alert['severity']}] {alert['alert_type']}: {alert['feature_name']}")
else:
    alerts_data = []
    print("ℹ️ No data for alert generation")

## Step 3.3: Write Alerts to Delta

In [ ]:
if alerts_data:
    schema_alerts = StructType([
        StructField("alert_id", StringType(), False),
        StructField("feature_id", StringType(), False),
        StructField("feature_name", StringType(), False),
        StructField("workload", StringType(), True),
        StructField("alert_type", StringType(), False),
        StructField("severity", StringType(), False),
        StructField("message", StringType(), False),
        StructField("setting_name", StringType(), True),
        StructField("similarity_score", DoubleType(), True),
        StructField("days_since_release", IntegerType(), True),
        StructField("alert_date", TimestampType(), False),
        StructField("acknowledged", BooleanType(), False),
        StructField("acknowledged_date", TimestampType(), True),
        StructField("acknowledged_by", StringType(), True)
    ])
    
    df_alerts = spark.createDataFrame(alerts_data, schema=schema_alerts)
    
    print(f"🔄 Writing {len(alerts_data)} alerts to {table_name_alerts}...")
    
    try:
        if DeltaTable.isDeltaTable(spark, table_path_alerts):
            print("  → Appending to existing table...")
            df_alerts.write.format("delta").mode("append").save(table_path_alerts)
        else:
            print("  → Creating new table...")
            df_alerts.write.format("delta").mode("overwrite").save(table_path_alerts)
        
        final_count = spark.read.format("delta").load(table_path_alerts).count()
        print(f"✅ Total alerts: {final_count}")
        
    except Exception as e:
        print(f"❌ Error writing alerts: {e}")
        raise
else:
    print("ℹ️ No new alerts to write")

---
# Summary & Statistics

In [ ]:
print("\n" + "=" * 70)
print("📊 FEATURE TRACKING SUMMARY")
print("=" * 70)

# Feature Releases Stats
df_roadmap = spark.read.format("delta").load(table_path)
total_features = df_roadmap.count()
preview_features = df_roadmap.filter(F.col("is_preview") == True).count()
planned_features = df_roadmap.filter(F.col("is_planned") == True).count()
shipped_features = df_roadmap.filter(F.col("is_shipped") == True).count()

print(f"\n🔸 Feature Releases Roadmap:")
print(f"   Total features: {total_features}")
print(f"   Preview features: {preview_features}")
print(f"   Planned (roadmap): {planned_features}")
print(f"   Shipped: {shipped_features}")

# Active Previews Stats
if matches:
    print(f"\n🔸 Activated Preview Features:")
    print(f"   Total activated: {len(matches)}")
    high_conf = sum(1 for m in matches if m["similarity_score"] >= 0.7)
    med_conf = sum(1 for m in matches if 0.5 <= m["similarity_score"] < 0.7)
    low_conf = sum(1 for m in matches if m["similarity_score"] < 0.5)
    print(f"   High confidence (≥0.7): {high_conf}")
    print(f"   Medium confidence (0.5-0.7): {med_conf}")
    print(f"   Low confidence (<0.5): {low_conf}")

# Alerts Stats
if alerts_data:
    print(f"\n🔸 Alerts Generated:")
    print(f"   New alerts: {len(alerts_data)}")
    info_alerts = sum(1 for a in alerts_data if a["severity"] == SEVERITY_INFO)
    warn_alerts = sum(1 for a in alerts_data if a["severity"] == SEVERITY_WARNING)
    crit_alerts = sum(1 for a in alerts_data if a["severity"] == SEVERITY_CRITICAL)
    print(f"   Info: {info_alerts} | Warning: {warn_alerts} | Critical: {crit_alerts}")

# Workload Breakdown
print(f"\n🔸 Top Workloads (by feature count):")
df_roadmap.groupBy("workload").count().orderBy(F.desc("count")).show(5, truncate=False)

print("\n" + "=" * 70)
print("✅ LOAD FEATURE TRACKING - COMPLETED")
print("=" * 70)
print("\n💡 View results in:")
print("   - vw_roadmap_upcoming (planned features)")
print("   - vw_active_preview_features (enabled previews)")
print("   - vw_critical_alerts (unacknowledged alerts)")
print("\n" + "=" * 70)